In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from datetime import datetime, timedelta

In [2]:
class Dataset:
    def __init__(self):
        self.df = None
    def append(self, df):
        if self.df is None:
            self.df = df
        else:
            self.df = pd.concat([self.df,df], ignore_index=True)

In [3]:

# Name, Entry TIme, Entry PRice, QTY, Exit Time, Exit Price

class Backtest:
    def __init__(self):
        self.columns = ['Equity Name', 'Trade', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'Quantity', 'Position Size', 'PNL', '% PNL']
        self.backtesting = pd.DataFrame(columns=self.columns)

    def buy(self, equity_name, entry_time, entry_price, qty):
        self.trade_log = dict(zip(self.columns, [None] * len(self.columns)))
        self.trade_log['Trade'] = 'Long Open'
        self.trade_log['Quantity'] = qty
        self.trade_log['Position Size'] = round(self.trade_log['Quantity'] * entry_price, 3)
        self.trade_log['Equity Name'] = equity_name
        self.trade_log['Entry Time'] = entry_time
        self.trade_log['Entry Price'] = round(entry_price, 2)

    def sell(self, exit_time, exit_price, exit_type, charge):
        self.trade_log['Trade'] = 'Long Closed'
        self.trade_log['Exit Time'] = exit_time
        self.trade_log['Exit Price'] = round(exit_price, 2)
        self.trade_log['Exit Type'] = exit_type
        self.trade_log['PNL'] = round((self.trade_log['Exit Price'] - self.trade_log['Entry Price']) * self.trade_log['Quantity'] - charge, 3)
        self.trade_log['% PNL'] = round((self.trade_log['PNL'] / self.trade_log['Position Size']) * 100, 3)
        self.trade_log['Holding Period'] = exit_time - self.trade_log['Entry Time']
        # self.backtesting = self.backtesting.append(self.trade_log, ignore_index=True)
        self.backtesting.loc[len(self.backtesting)] = self.trade_log
    def stats(self):
        df = self.backtesting
        parameters = ['Total Trade Scripts', 'Total Trade', 'PNL',  'Winners', 'Losers', 'Win Ratio','Total Profit', 'Total Loss', 'Average Loss per Trade', 'Average Profit per Trade', 'Average PNL Per Trade', 'Risk Reward']
        total_traded_scripts = len(df['Equity Name'].unique())
        total_trade = len(df.index)
        pnl = df.PNL.sum()
        winners = len(df[df.PNL > 0])
        loosers = len(df[df.PNL <= 0])
        win_ratio = str(round((winners/total_trade) * 100, 2)) + '%'
        total_profit = round(df[df.PNL > 0].PNL.sum(), 2)
        total_loss  = round(df[df.PNL <= 0].PNL.sum(), 2)
        average_loss_per_trade = round(total_loss/loosers, 2)
        average_profit_per_trade = round(total_profit/winners, 2)
        average_pnl_per_trade = round(pnl/total_trade, 2)
        risk_reward = f'1:{-1 * round(average_profit_per_trade/average_loss_per_trade, 2)}'
        data_points = [total_traded_scripts,total_trade,pnl,winners, loosers, win_ratio, total_profit, total_loss, average_loss_per_trade, average_profit_per_trade, average_pnl_per_trade, risk_reward]
        data = list(zip(parameters,data_points ))
        print(tabulate(data, ['Parameters', 'Values'], tablefmt='psql'))
        

In [4]:
from datetime import datetime, timedelta
today_date = datetime.now()- timedelta(days=7)
stocks_to_buy =[]
# dt=Dataset()
def GoldenCrossverSignal(name):
    path = f'./Data/{name}.csv'
    data = pd.read_csv(path, parse_dates=['Date'], ) #index_col='Date'
    data['Date'] = pd.to_datetime(data['Date'])
    
    data['Prev_Close'] = data.Close.shift(1)
    data['20_SMA'] = data.Prev_Close.rolling(window=20, min_periods=1).mean()
    data['50_SMA'] = data.Prev_Close.rolling(window=50, min_periods=1).mean()
    data['20_SMA_diff_7'] =  data['20_SMA'] - data['20_SMA'].shift(7)
    data['20_SMA_diff_14'] =  data['20_SMA'] - data['20_SMA'].shift(14)
    data['50_SMA_diff_14'] = data['50_SMA'] - data['50_SMA'].shift(14) 
    data['50_SMA_diff_7'] = data['50_SMA'] - data['50_SMA'].shift(7) 
    data['50_SMA_diff'] =  data['50_SMA'].shift(-14) - data['50_SMA']
    
    # data = data.iloc[-x:y]
    data['Signal'] = 0
    data['Signal'] = np.where((data['50_SMA_diff'] > 0), 1, 0)
    data['Position'] = data.Signal.diff()
    prev=0
    data['AlreadyBought'] =0
    bought = 0
    for i , val in data['Position'].iteritems():
        if val ==1 or val == -1:
            diff = i-prev
            mid = diff//2
            if mid > prev and  mid < i:   data.at[mid,'Position'] = 2
            prev = i
        if val == 1:
            bought = 1
        if val == -1:
            bought = 0
        data.at[i,'AlreadyBought'] = bought
    
        
    # data.to_csv("test33.csv")
    df_pos = data[(data['Position'] == 1) | (data['Position'] == -1) | (data['Position'] == 2)].copy()
    dt.append(df_pos)
    # df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
    
    # print(df_pos.columns)
    # Filter the DataFrame based on the conditions
    
    

    return  df_pos,data,name
# sty='3PLAND'
# GoldenCrossverSignal(sty)

In [32]:
# try:
#         filtered_df = df_pos.loc[(df_pos["Date"] >= seven_days_ago)]
#         if len(filtered_df) : 
#             # print(filtered_df['Position'].iat[-1] )
#             if   filtered_df['Position'].iat[-1] == "Buy":
#                 stocks_to_buy.append(name)
#                 sig_date = filtered_df['Date'].iloc[0]
#                 z= sig_date - pd.Timedelta(days=0)
#                 old_price = filtered_df.loc[filtered_df['Date'] == z]['Close'].iloc[0]
#                 cur_price = data.loc[data['Date'] >= today_date]['Close'].iloc[0]
#                 perc = ((cur_price-old_price)/old_price)*100
#                 print(name,perc,sig_date)
#     except Exception as e:
#         print(f"error {name} , {e}")
delta_date = datetime.now() - timedelta(days=365)

def list_buy(df,data,name):
    # print(df.columns)
    filtered_df = df.loc[(df["Date"] >= delta_date) & (df["Position"] == "Buy")]
    if len(filtered_df) == 0 : return
    # Find the latest date of BUY
    latest_buy = filtered_df['Date'].max()
    dff =df.sort_values(by="Date", ascending=False)
    dff.to_csv(f"./filtered/{name}.csv")
    # Check 
    sell_date = df[(df['Position'] == 'Sell') & (df['Date'] > delta_date)]['Date'].max()
     
    if latest_buy > sell_date :
        latest_date = data["Date"].max()
        cur_p = data.loc[data["Date"]==latest_date]["Close"].iloc[0]
        buy_p = data.loc[data["Date"]==latest_buy]["Close"].iloc[0]
        percent = ((cur_p - buy_p ) / buy_p ) *100
        print(name, buy_p, cur_p, percent, latest_buy)


In [5]:
from tqdm import tqdm

In [9]:
stocks = pd.read_csv('.\EQUITY_L.csv')
scripts = stocks["SYMBOL"]
# scripts=["IEX"]
for stock in tqdm(scripts[:]):
    filtered_df, data, name = GoldenCrossverSignal(stock)
    # list_buy(filtered_df,data,name)
    

  0%|          | 0/1935 [00:00<?, ?it/s]

 32%|███▏      | 625/1935 [00:21<00:44, 29.13it/s]C:\Users\Shubham Soni\AppData\Local\Temp\ipykernel_15748\2612810024.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df,df], ignore_index=True)
100%|██████████| 1935/1935 [01:09<00:00, 28.00it/s]


In [10]:
dt.df.to_csv("dataset_stock.csv")

In [11]:
dt.df.describe

<bound method NDFrame.describe of             Date         Open         High          Low        Close  \
0     2014-01-17    31.000000    31.000000    30.500000    30.650000   
1     2014-02-07    32.200001    32.900002    31.350000    32.549999   
2     2014-03-20    30.299999    30.450001    30.200001    30.250000   
3     2014-07-04    35.299999    35.799999    35.000000    35.200001   
4     2014-12-16    30.400000    30.400000    30.000000    30.100000   
...          ...          ...          ...          ...          ...   
67341 2023-05-17  1565.000000  1565.000000  1475.050049  1481.150024   
67342 2023-07-24  1476.949951  1491.349976  1459.250000  1474.400024   
67343 2023-10-19  1569.500000  1569.500000  1547.699951  1555.250000   
67344 2023-12-06  1540.000000  1559.949951  1538.000000  1549.849976   
67345 2023-12-28  1653.949951  1665.500000  1634.099976  1643.150024   

         Adj Close  Volume   Prev_Close       20_SMA       50_SMA  \
0        29.225180  123045    30

In [33]:
filtered_df, data, name = GoldenCrossverSignal(sty)
list_buy(filtered_df,data,sty)

3PLAND 30.450000762939453 36.25 19.047616064823544 2024-01-09 00:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.metrics import accuracy_score ,mean_squared_error ,classification_report
# df = df.dropna()
# Sample DataFrame with features and 'Label' as the last column
# Assuming 'Label' is the last column, separate features and labels
X = data_new.iloc[:, :-1]  # Features
y = data_new.iloc[:, -1]   # Labels


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=50, random_state=42, max_depth = 15)
# clf = LogisticRegression()

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict_proba(X_test)[:,0]

y_pred = [0 if x > 0.8 else  1 for x in y_pred ]

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
classification_report_output = classification_report(y_test, y_pred)
print('Classification Report:')
print(classification_report_output)
# Plotting the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=[ 'No Signal', 'Buy Signal'],
            yticklabels=['No Signal', 'Buy Signal'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
